In [ ]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds

#MNIST data set
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #0-255 to 0-1
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#creating the model
modelo = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) 
])

#Compiling the model
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 32

#shuffling
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#training
historial = modelo.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/5
1875/1875 [==============================] - 69s 33ms/step - loss: 0.3010 - accuracy: 0.9064
Epoch 2/5
1875/1875 [==============================] - 62s 33ms/step - loss: 0.1156 - accuracy: 0.9651
Epoch 3/5
1875/1875 [==============================] - 61s 33ms/step - loss: 0.0941 - accuracy: 0.9718
Epoch 4/5
1875/1875 [==============================] - 61s 33ms/step - loss: 0.0753 - accuracy: 0.9775
Epoch 5/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0637 - accuracy: 0.9803


In [ ]:
#exporting the model to h5
modelo.save('numeros.h5')

In [ ]:
!ls

numeros.h5  sample_data


In [ ]:
!pip install tensorflow js

     |████████████████████████████████| 237 kB 10.4 MB/s 
     |████████████████████████████████| 114 kB 55.5 MB/s 
  Created wheel for js: filename=js-1.0-py3-none-any.whl size=2907 sha256=e98fdaefd437875698f7c80fd81abc4a955c8d9c8aa9b6af32057d62f7eaaebc
  Stored in directory: /root/.cache/pip/wheels/bf/92/56/6597c8f32980ee64af7e864796e8d7423eb67eafff5415a4ae
  Created wheel for fanstatic: filename=fanstatic-1.1-py3-none-any.whl size=31971 sha256=fc9862672c04d6a1686b73282751160f2e62a9a8560c13f878567fe441393860
  Stored in directory: /root/.cache/pip/wheels/6f/2e/4f/5965d572ae25745ee3586ceca90a98463afcd05f0298e37d82
  Created wheel for shutilwhich: filename=shutilwhich-1.1.0-py3-none-any.whl size=2781 sha256=f0a1ca0b6fcf8d105b52010566b999997715883842798437d77e35d7b14c77e1
  Stored in directory: /root/.cache/pip/wheels/4c/c0/ce/5a47f35186acbe73e38811df30daaf798c81a7599bb2d5995a
Successfully built js fanstatic shutilwhich


In [ ]:
#folder with output files
!mkdir carpeta_salida

In [ ]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

/bin/bash: tensorflowjs_converter: command not found


In [ ]:
!ls carpeta_salida

In [ ]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html